In [1]:
!pip install streamlit pandas pyttsx3 SpeechRecognition requests python-dotenv google-api-python-client openpyxl

In [2]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2

In [31]:
code = """
import streamlit as st
import pandas as pd
import sqlite3
import speech_recognition as sr
import pyttsx3
import os
import threading
import requests
from googleapiclient.discovery import build
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env
load_dotenv()

# --- API Keys ---
openai_api_key = os.getenv("your_openai_api_key")       # Your OpenAI key
huggingface_key = os.getenv("your_youtube_api_key")     # Your Hugging Face key
youtube_api_key = os.getenv("your_huggingface_api_key")     # Your YouTube key

# --- Set up OpenAI client ---
client = OpenAI(api_key="your_openai_api_key")

# --- SQLite DB setup ---
conn = sqlite3.connect('chat_history.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS conversations (role TEXT, content TEXT)''')
conn.commit()

# --- Text-to-Speech function ---
def speak(text):
    def _speak():
        engine = pyttsx3.init()
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# --- Save and fetch chat history ---
def save_conversation(role, content):
    c.execute('INSERT INTO conversations (role, content) VALUES (?, ?)', (role, content))
    conn.commit()

def fetch_conversation_history():
    c.execute('SELECT role, content FROM conversations ORDER BY ROWID DESC LIMIT 10')
    return [{"role": r, "content": c} for r, c in reversed(c.fetchall())]

# --- Code Generation (supports all languages based on prompt) ---
def generate_code_from_input(user_input):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # or "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You are an expert developer. Generate clean and runnable code in the language requested."},
                {"role": "user", "content": user_input}
            ],
            temperature=0.5
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Error generating code: {e}"

# --- Hugging Face Text Generation ---
def generate_response(user_input):
    headers = {"Authorization": f"Bearer {"your_huggingface_api_key"}"}
    payload = {
        "inputs": user_input,
        "parameters": {
            "temperature": 0.7,
            "max_new_tokens": 100,
            "return_full_text": False
        }
    }

    response = requests.post(
        "https://huggingface.co/deepseek-ai/deepseek-coder-6.7b-instruct",
        headers=headers,
        json=payload
    )

    if response.status_code == 200:
        result = response.json()
        return result[0]['generated_text']
    else:
        return "⚠️ Sorry, I couldn't generate a response right now."

# --- Hugging Face Sentiment Analysis ---
def huggingface_sentiment(text):
    headers = {"Authorization": f"Bearer {"Your_huggingface_api_key"}"}
    response = requests.post(
        "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english",
        headers=headers,
        json={"inputs": text}
    )
    return response.json()

# --- YouTube Search ---
def search_youtube(query):
    youtube = build('youtube', 'v3', developerKey="your_youtube_api_key")
    request = youtube.search().list(q=query, part='snippet', type='video', maxResults=3)
    response = request.execute()
    return response['items']

# --- Streamlit UI ---
st.set_page_config(page_title="CodeBot", layout="centered")
st.title("🤖 CodeBot - AI Developer Assistant")
st.write("Ask me to generate code, answer questions, or analyze your input!")

# Chat history display
messages = fetch_conversation_history()
for msg in messages:
    st.markdown(f"**{'🧑 You' if msg['role'] == 'user' else '🤖 Bot'}:** {msg['content']}")

# File Upload
uploaded_file = st.file_uploader("📁 Upload CSV or Excel file", type=["csv", "xlsx"])
if uploaded_file:
    df = pd.read_csv(uploaded_file) if uploaded_file.name.endswith(".csv") else pd.read_excel(uploaded_file)
    st.write(df)

# Voice or Text input
user_input = None
if st.button("🎙️ Speak"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("🎧 Listening...")
        audio = recognizer.listen(source)
    try:
        user_input = recognizer.recognize_google(audio)
        st.write(f"🗣️ You said: {user_input}")
    except Exception as e:
        st.error(f"Voice input error: {e}")
else:
    user_input = st.text_input("💬 Type your question or code request:")

# If user provided input
if user_input:
    save_conversation("user", user_input)

    with st.spinner("🤔 Thinking..."):
        # AI code generation
        code_output = generate_code_from_input(user_input)
        bot_reply = generate_response(user_input)
        yt_results = search_youtube(user_input)
        sentiment = huggingface_sentiment(user_input)

        # Display output
        st.subheader("📄 Generated Code")
        st.code(code_output)

        st.subheader("💡 AI Reply")
        st.markdown(f"**🤖 Bot:** {bot_reply}")
        speak(bot_reply)

        st.subheader("🧠 Sentiment Analysis")
        st.json(sentiment)

        st.subheader("🎥 Related YouTube Videos")
        for vid in yt_results:
            video_url = f"https://www.youtube.com/watch?v={vid['id']['videoId']}"
            st.markdown(f"- [{vid['snippet']['title']}]({video_url})")

        save_conversation("assistant", bot_reply)

"""

In [33]:
with open("Ds_ml_chatbotai1.py", "w", encoding="utf-8") as f:
    f.write(code)

print("File saved as Ds_ml_chatbotai1.py ✅")

File saved as Ds_ml_chatbotai1.py ✅


In [37]:
# how can run this code in tour terminal 
# in these AI run you can first purchase the openai packages
# you can also change the mode name according to you need
#in terminal do 
# import pyaudio for speak to search
# cd " name where your file saved"
# dir to cheack file
# streamlit run your file saving name.py
# import any thing according to you in your terminal window to run this AI code